In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
from pathlib import Path

In [5]:
NORMAL_FOLDER = "csv_data/normal"
DEFECT_FOLDER = "csv_data/defect"
FRAME = 500

In [6]:
def load_and_process_data(file_path, label, window_size):
    df = pd.read_csv(file_path)
    signal = df['vibration'].values
    
    # Нормализация данных (важный шаг для нейросетей)
    scaler = StandardScaler()
    signal_scaled = scaler.fit_transform(signal.reshape(-1, 1)).flatten()

    X, y = [], []
    for i in range(0, len(signal_scaled) - window_size, window_size):
        window = signal_scaled[i : i + window_size]
        X.append(window)
        y.append(label)
        
    return np.array(X), np.array(y)

In [10]:
def create_1d_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        
        Conv1D(filters=64, kernel_size=5, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        
        Flatten(), 
        
        Dense(100, activation='relu'),
        Dense(1, activation='sigmoid') 
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [7]:
def create_cnn_1d_2l(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=5, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
from pathlib import Path
import numpy as np

X_normals_list = []
y_normals_list = []

for file in Path(NORMAL_FOLDER).iterdir():
    if file.is_file():
    
        X_normal, y_normal = load_and_process_data(file, 0, FRAME)
        X_normals_list.append(X_normal)
        y_normals_list.append(y_normal)

X_defects_list = []
y_defects_list = []

for file in Path(DEFECT_FOLDER).iterdir():
    if file.is_file():
        X_defect, y_defect = load_and_process_data(file, 1, FRAME)
        X_defects_list.append(X_defect)
        y_defects_list.append(y_defect)

if X_normals_list and X_defects_list:
    X_normals = np.concatenate(X_normals_list, axis=0)
    y_normals = np.concatenate(y_normals_list, axis=0)
    
    X_defects = np.concatenate(X_defects_list, axis=0)
    y_defects = np.concatenate(y_defects_list, axis=0)
    
    X = np.concatenate((X_normals, X_defects), axis=0)
    y = np.concatenate((y_normals, y_defects), axis=0)
    
    print(X.shape)
    print(y.shape)
else:
    print("Не удалось загрузить данные. Списки пусты.")



(6311, 500)
(6311,)


In [12]:
X = X.reshape((X.shape[0], X.shape[1], 1))

print(X.shape)

# Шаг 3: Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Форма обучающих данных (X_train): {X_train.shape}")
print(f"Форма тестовых данных (X_test): {X_test.shape}")

# Шаг 4: Создаем и обучаем модель
print("\nСоздание и обучение модели...")
model = create_cnn_1d_2l(input_shape=(FRAME, 1))
model.summary()

history = model.fit(
    X_train, y_train,
    epochs=3,
    batch_size=32,
    validation_data=(X_test, y_test)

)



(6311, 500, 1)
Форма обучающих данных (X_train): (5048, 500, 1)
Форма тестовых данных (X_test): (1263, 500, 1)

Создание и обучение модели...


c:\Users\scree\Desktop\practic\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 496, 32)        │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 248, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 244, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 122, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 7808)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       499,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 510,337 (1.95 MB)

 Trainable params: 510,337 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9440 - loss: 0.1491 - val_accuracy: 1.0000 - val_loss: 8.0024e-05
Epoch 2/3
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 1.0000 - loss: 3.8910e-04 - val_accuracy: 1.0000 - val_loss: 4.2679e-06
Epoch 3/3
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 1.0000 - loss: 1.2298e-04 - val_accuracy: 1.0000 - val_loss: 1.4253e-06


In [13]:
print("\nОценка модели...")
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Точность модели на тестовых данных: {accuracy * 100:.2f}%")


Оценка модели...
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 1.4080e-06 
Точность модели на тестовых данных: 100.00%


In [14]:
model.save("bearing_defect_model_v4.h5")
print(f"\nМодель сохранена в файл: {"bearing_defect_model_v4.h5"}")


Модель сохранена в файл: bearing_defect_model_v4.h5


In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# --- Конфигурация ---
WINDOW_SIZE = 500
MODEL_FILE = "bearing_defect_model_v3.h5"
NEW_FILE = "csv_self_data/defect/20Hz_accelerometer.csv"  # замените на имя вашего файла

# --- Загрузка и обработка новых данных ---
def load_and_prepare(file_path, window_size):
    df = pd.read_csv(file_path)
    signal = df['vibration'].astype(np.float32).values

    # Нормализация (на новых данных обучаем scaler заново)
    scaler = StandardScaler()
    signal_scaled = scaler.fit_transform(signal.reshape(-1, 1)).flatten()

    X = []
    for i in range(0, len(signal_scaled) - window_size + 1, window_size):
        window = signal_scaled[i : i + window_size]
        if len(window) == window_size:
            X.append(window)
    X = np.array(X)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    return X

# --- Загрузка модели ---
model = tf.keras.models.load_model(MODEL_FILE)

# --- Подготовка данных ---
X_new = load_and_prepare(NEW_FILE, WINDOW_SIZE)

# --- Предсказание ---
y_pred_prob = model.predict(X_new)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# --- Вывод результатов ---
for i, (prob, label) in enumerate(zip(y_pred_prob, y_pred)):
    print(f"Окно {i+1}: вероятность дефекта = {prob[0]:.2f}, {'ДЕФЕКТ' if label else 'Норма'}")

# Если хотите получить общее решение по всему файлу:
if np.mean(y_pred) > 0.5:
    print("\nИТОГ: Скорее всего, дефект присутствует в файле.")
else:
    print("\nИТОГ: Дефект не обнаружен.")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Окно 1: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 2: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 3: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 4: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 5: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 6: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 7: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 8: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 9: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 10: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 11: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 12: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 13: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 14: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 15: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 16: вероятность дефекта = 0.31, Норма
Окно 17: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 18: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 19: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 20: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 21: вероятность дефекта = 1.00, ДЕФЕКТ
Окно 22: вероятность дефекта = 1.00, ДЕФЕКТ
Окно